In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

# Assume df is your DataFrame containing the pressure data and labels
# df['label'] should be 'Okay' or 'Fault'
# df['prefilling_values'], df['filling_values'], df['stabilizing_values'] should be lists of pressure values

# Convert labels to numerical format
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

# Create sequences (example: combine pre-filling, filling, stabilizing values)
sequences = df.apply(lambda row: row['prefilling_values'] + row['filling_values'] + row['stabilizing_values'], axis=1)
sequences = sequences.tolist()

# Padding sequences to ensure consistent length
from keras.preprocessing.sequence import pad_sequences
max_seq_length = max(len(seq) for seq in sequences)
sequences_padded = pad_sequences(sequences, maxlen=max_seq_length, padding='post', dtype='float32')

# Split data
X = np.array(sequences_padded)
y = df['label'].values
y = to_categorical(y)  # One-hot encode the labels

X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, random_state=42)

# Model Design
model = Sequential()
model.add(LSTM(50, input_shape=(max_seq_length, 1), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Reshape data for LSTM input
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Training
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Evaluation
y_test_pred = model.predict(X_test)
y_test_pred_classes = np.argmax(y_test_pred, axis=1)
y_test_true_classes = np.argmax(y_test, axis=1)

print(classification_report(y_test_true_classes, y_test_pred_classes))
print(confusion_matrix(y_test_true_classes, y_test_pred_classes))




ModuleNotFoundError: No module named 'tensorflow'